Project 2: Market Basket Analysis & Cross-Selling
Dataset: Retail Product Transaction (30k rows)

In [39]:
import pandas as pd

df = pd.read_csv("data/products.csv")

print(df.info())
 
print(df.head(5))
 
print(df.tail(5))
 
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TransactionID  30000 non-null  int64 
 1   CustomerID     30000 non-null  object
 2   Products       30000 non-null  object
 3   Timestamp      30000 non-null  object
dtypes: int64(1), object(3)
memory usage: 937.6+ KB
None
   TransactionID CustomerID  \
0              1       C546   
1              2       C385   
2              3       C292   
3              4       C863   
4              5       C171   

                                                                                      Products  \
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
1                                                   Onion, Juice, Flatbread with Meat, Chicken   
2                                                      Egg, Flatbread with Meat, Banana, Pizza  

In [40]:
df["CustomerID"].nunique()

1000

In [41]:
df["Products"].nunique()

29889

dataset has 1000 customers, and 29889 product combinations

Preparing Data for Apriori- binary matrix

In [42]:
df["ProductList"] = df["Products"].str.split(",") 
df["ProductList"] = df["ProductList"].apply(lambda x: [i.strip() for i in x])

In [43]:
print(df[["TransactionID", "ProductList"]].head())

   TransactionID  \
0              1   
1              2   
2              3   
3              4   
4              5   

                                                                                     ProductList  
0  [Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream]  
1                                                   [Onion, Juice, Flatbread with Meat, Chicken]  
2                                                      [Egg, Flatbread with Meat, Banana, Pizza]  
3                                  [Ice Cream, Soda, Orange, Potato, Cereal, Chocolate, Cracker]  
4                                     [Ice Cream, Soap, Shampoo, Chicken, Banana, Beans, Cheese]  


In [44]:
exploded = df.explode("ProductList")

In [45]:
print(exploded.head(10))

   TransactionID CustomerID  \
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
0              1       C546   
1              2       C385   

                                                                                      Products  \
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, Cucumber, Rice, Ice Cream   
0  Dish Sponge, Flatbread with Meat, Chips, Orange, Butter, Sausage, C

basket matrix

In [46]:
basket = exploded.groupby(["TransactionID", "ProductList"])["ProductList"].count().unstack().fillna(0)
basket = basket.astype(bool).astype(int)

In [47]:
print(basket.head())

ProductList    Apple  Banana  Beans  Bread  Butter  Cereal  Cheese  Chicken  \
TransactionID                                                                 
1                  0       0      0      0       1       0       0        0   
2                  0       0      0      0       0       0       0        1   
3                  0       1      0      0       0       0       0        0   
4                  0       0      0      0       0       1       0        0   
5                  0       1      1      0       0       0       1        1   

ProductList    Chickpeas  Chips  ...  Potato  Rice  Sausage  Shampoo  Soap  \
TransactionID                    ...                                         
1                      0      1  ...       0     1        1        0     0   
2                      0      0  ...       0     0        0        0     0   
3                      0      0  ...       0     0        0        0     0   
4                      0      0  ...       1     0      

In [48]:
basket = basket.astype(bool)

Apriori Algorithm

In [49]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

rules = rules.sort_values(by="confidence", ascending=False)


print(rules.head(10))


     antecedents consequents  antecedent support  consequent support  \
173       (Milk)    (Cereal)            0.162667            0.232733   
172     (Cereal)      (Milk)            0.232733            0.162667   
159  (Chocolate)    (Cereal)            0.160800            0.232733   
165   (Cucumber)    (Cereal)            0.159933            0.232733   
155     (Cheese)    (Cereal)            0.165267            0.232733   
129     (Butter)    (Cereal)            0.162033            0.232733   
182       (Soda)    (Cereal)            0.165800            0.232733   
175      (Onion)    (Cereal)            0.162400            0.232733   
156      (Chips)    (Cereal)            0.161700            0.232733   
185     (Tomato)    (Cereal)            0.162567            0.232733   

      support  confidence      lift  representativity  leverage  conviction  \
173  0.095067    0.584426  2.511141               1.0  0.057209    1.846283   
172  0.095067    0.408479  2.511141              

In [50]:
pd.set_option("display.max_colwidth", None)

In [51]:
print(f"Total rules generated: {len(rules)}")

Total rules generated: 724


In [52]:
rules_clean = rules[[
    'antecedents',
    'consequents',
    'support',
    'confidence',
    'lift'
]]


In [53]:
rules_clean['antecedents'] = rules_clean['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_clean['consequents'] = rules_clean['consequents'].apply(lambda x: ', '.join(list(x)))


C:\Users\JOSHUA\AppData\Local\Temp\ipykernel_24564\1257458078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rules_clean['antecedents'] = rules_clean['antecedents'].apply(lambda x: ', '.join(list(x)))
C:\Users\JOSHUA\AppData\Local\Temp\ipykernel_24564\1257458078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rules_clean['consequents'] = rules_clean['consequents'].apply(lambda x: ', '.join(list(x)))


In [54]:
print(rules_clean.head(10))

    antecedents consequents   support  confidence      lift
173        Milk      Cereal  0.095067    0.584426  2.511141
172      Cereal        Milk  0.095067    0.408479  2.511141
159   Chocolate      Cereal  0.038933    0.242123  1.040344
165    Cucumber      Cereal  0.038600    0.241351  1.037026
155      Cheese      Cereal  0.039733    0.240420  1.033026
129      Butter      Cereal  0.038733    0.239045  1.027122
182        Soda      Cereal  0.039400    0.237636  1.021064
175       Onion      Cereal  0.038433    0.236658  1.016865
156       Chips      Cereal  0.038233    0.236446  1.015953
185      Tomato      Cereal  0.038300    0.235596  1.012299


In [55]:
rules.to_csv("data/market_basket_rules.csv", index=False)

In [56]:
frequent_itemsets.to_csv("data/frequent_itemsets.csv", index=False)

In [57]:
import pandas as pd

fi = pd.read_csv("data/frequent_itemsets.csv")


In [58]:
fi["itemsets"] = fi["itemsets"].apply(lambda x: x.replace("frozenset(", "").replace("})", "}").replace("{", "").replace("}", "").replace("'", ""))

In [59]:
print(fi.head())

    support itemsets
0  0.160467    Apple
1  0.163033   Banana
2  0.164967    Beans
3  0.162333    Bread
4  0.162033   Butter


In [60]:
print(fi.tail())

      support            itemsets
815  0.027500   Water, Strawberry
816  0.026567  Strawberry, Yogurt
817  0.025333       Water, Tomato
818  0.024967      Tomato, Yogurt
819  0.027100       Water, Yogurt


In [61]:
fi.to_csv("data/frequent_itemsets_clean.csv", index=False)